In [10]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sp
import seaborn as sns
import numpy as np

#create list of unique columns
ordered = []
allcols = passing.columns.tolist() + rushing.columns.tolist() + receiving.columns.tolist()
for c in allcols:
    if c in ordered:
        pass
    else:
        ordered.append(c)


In [11]:
#import data and fix 1 column name
passing = pd.read_csv('Data/Career_Stats_Passing.csv')
rushing = pd.read_csv('Data/Career_Stats_Rushing.csv')
receiving = pd.read_csv('Data/Career_Stats_Receiving.csv')
receiving = receiving.rename(columns={'Yards Per Game':'Receiving Yards Per Game'})

#merge dfs
df1 = passing.merge(rushing,'outer',on=['Player Id','Name','Position','Year','Team','Games Played'])
df= df1.merge(receiving,'outer',on=['Player Id','Name','Position','Year','Team','Games Played'])

#organize columns into objects, floats, and integers. make passer rating separate.
objects = ['Player Id','Name','Position','Team']
float_cols = ['Completion Percentage','Pass Attempts Per Game','Passing Yards Per Attempt',
             'Passing Yards Per Game', 'Percentage of TDs per Attempts','Int Rate',
              'Rushing Attempts Per Game','Yards Per Carry','Rushing Yards Per Game',
             'Percentage of Rushing First Downs','Yards Per Reception',
              'Receiving Yards Per Game']
int_cols = df.columns.drop(objects + float_cols + ['Passer Rating']).tolist()

#get rid of unwanted text
#figure out which columns are numbers already, and which are strings
already_num = ['Year','Games Played','Pass Attempts Per Game','Passer Rating',
               'Rushing Attempts Per Game','Receiving Yards Per Game']

#remove commas
#remove text 'T'
#change '--' to 0
for c in df.columns.drop(objects + already_num):
    df[c] = df[c].str.replace(',','')
    df[c] = df[c].str.replace('T','')
    df.loc[df[c]=='--',c]=0

#fill na with 0
df = df.fillna(0)

#change dtypes
df[int_cols]= df[int_cols].astype(int)
df[float_cols]= df[float_cols].astype(float)

#fix fumbles
df['Fumbles'] = df['Fumbles_x'] + df['Fumbles_y']
df=df.drop(['Fumbles_x','Fumbles_y'],axis=1)
df= df.loc[df['Year']>1900]
df = df.loc[df['Games Played']>0]
#add seasons
df['Seasons']=1
df.shape

(21497, 45)

In [12]:
### Create functions that will recalculate columns which are formulas

#column formulas dictionary
#the key is the column, and the values are the the numerator and denominator
recalc_dict = {'Completion Percentage':('Passes Completed','Passes Attempted'),
                     'Pass Attempts Per Game':('Passes Attempted','Games Played'),
                     'Passing Yards Per Attempt':('Passing Yards','Passes Attempted'),
                     'Passing Yards Per Game':('Passing Yards','Games Played'),
                     'Percentage of TDs per Attempts':('TD Passes','Passes Attempted'),
                     'Int Rate':('Ints','Passes Attempted'),
                     'Rushing Attempts Per Game': ('Rushing Attempts','Games Played'),
                     'Yards Per Carry':('Rushing Yards','Rushing Attempts'),
                     'Rushing Yards Per Game':('Rushing Yards','Games Played'),
                     'Percentage of Rushing First Downs':('Rushing First Downs','Rushing Attempts'),
                     'Yards Per Reception':('Receiving Yards','Receptions'),
                     'Receiving Yards Per Game':('Receiving Yards','Games Played')}

# make function to recaluclate float columns
def recalculate(df,dictionary):
    columns = dictionary.keys()
    for c in columns:
        numerator = dictionary[c][0]
        denominator = dictionary[c][1]
        df[c] = 0
        df.loc[df[denominator]>200,c] = df[numerator] / df[denominator]
        df.loc[df[c].isnull(),c]= 0


#function to recalculate passer rating
def recalc_pr(df):
    a = (df['Completion Percentage'] - .3) * 5
    b = (df['Passing Yards Per Attempt'] - 3) * .25
    c = (df['Percentage of TDs per Attempts'] * 20)
    d = 2.375 - (df['Int Rate'] * 25)
    df['Passer Rating'] = 0
    df.loc[df['Passes Attempted']>200,'Passer Rating'] = ((a + b + c + d)/6)*100
    df.loc[df['Passer Rating'].isnull(),'Passer Rating']=0


In [13]:
## get positions
def get_position(df):
    for row in np.arange(0,len(df)):
        passing = df.loc[row,'Passing Yards']
        rushing = df.loc[row,'Rushing Yards']
        receiving = df.loc[row,'Receiving Yards']
        highest = max(passing,rushing,receiving)
        if highest == passing:
            df.loc[row,'Position'] = 'QB'
        elif highest == rushing:
            df.loc[row,'Position'] = 'TB'
        elif highest == receiving:
            df.loc[row,'Position'] = 'WR/TE'


In [14]:
#function that fixes both floats and passer rating
def recalc(df):
    get_position(df)
    recalculate(df,recalc_dict)
    recalc_pr(df)

In [15]:
#function for inflation
inf_dict = {}
def inflation(df,groupby,col1):
    #make new column and set default to 0
    new_col = col1 + ' inf' 
    new_colmax = col1 + ' infmax'
    new_column = col1 + ' inf%'
    newnumber = col1 + ' adj'
    #get the individual group we are checking for inflation
    groups = df[groupby].unique()
    #list of all top15s
    top15s = []
    for g in groups:
        #get the group we want
        thegroup = df.loc[df[groupby]==g].sort_values(col1,ascending=False).reset_index()
        #find the mean of top 15 ppl and add to list
        top15mean = thegroup.loc[0:14,col1].mean()
        top15s.append(top15mean)
        #set all people from that group to that mean
        df.loc[df[groupby]==g,new_col] = top15mean
    
        
    #set max column
    df[new_colmax] = max(top15s)
    #set inf%
    df[new_column] = df[new_colmax] / df[new_col]
    #make column for inflation adjusted number
    df[newnumber] = df[col1] * df[new_column]
    #drop the unneeded columns
    df = df.drop([new_colmax,new_col],axis=1)
    return df

In [16]:
def fix_inf(old_df,new_df,groupby,column):
    diction = old_df[[groupby,column + ' inf%']].groupby(groupby).mean()
    uniques = diction.index
    for u in uniques:
        new_df.loc[new_df[groupby]==u,column + ' inf%'] = diction.loc[u].mean()
    new_df[column + ' adj'] = new_df[column] * new_df[column + ' inf%']
    new_df = new_df.drop([column + ' inf%',column],axis=1)
    return new_df

In [17]:
#Find min and max years
def min_and_max(df,column):
    minmax = pd.DataFrame()
    players = df['Name'].unique()
    minmax['Name'] = players
    for p in players:
        first = min(df.loc[df['Name']==p,column])
        last = max(df.loc[df['Name']==p,column])
        minmax.loc[minmax['Name']==p,'First ' + column] = first
        minmax.loc[minmax['Name']==p,'Last ' + column] = last
    return minmax
minmax = min_and_max(df,'Year')

In [18]:
# Merge min and max years
df = df.merge(minmax)
df['First Year'] = df['First Year'].astype(int)
df['Last Year'] = df['Last Year'].astype(int)
df['5Year'] = df['Year'] // 5 *5
df.shape

(21497, 48)

In [19]:
df['5Year'] = df['Year'] // 5 *5
df = inflation(df,'5Year','Passing Yards')
df = inflation(df,'5Year','Rushing Yards')
df = inflation(df,'5Year','TD Passes')
df = inflation(df,'5Year','Rushing TDs')
df = inflation(df,'5Year','Receiving Yards')
df = inflation(df,'5Year','Receiving TDs')

### Get Hall of Famers

In [20]:
#make df for HOF players
hof = pd.read_csv('Data/hof.csv')
#get only positions we want
hof['Position'] = hof['Position'].astype(str)
positions_wanted = ['RB','HB','TB','QB','TE','WR','FB']
hof['NewCol']= False
for r in np.arange(0,len(hof)):
    position = hof.loc[r,'Position']
    if position in positions_wanted:
        hof.loc[r,'NewCol']= True
hof = hof[hof['NewCol']==True]

#remove slashes
hof_players = pd.DataFrame(hof['Name'].str.split('\\',expand=True)[0])
#change to format Last Name, First Name
hof_players = hof_players[0].str.split(' ',expand=True)
for c in hof_players.columns:
    hof_players[c]=hof_players[c].astype(str)
hof_players.loc[hof_players[2]=='None',2]=''
hof_players['Name'] = hof_players[1] + hof_players[2] + ', ' + hof_players[0]
hof['Name'] = hof_players['Name']
#add hof column
hof_players['HOF']=1
#grab only columns we want
hof_players = hof_players[['Name','HOF']]
#merge hof players
df1 = df.merge(hof_players,'outer')
#separate out hof players who dont have stats in our previous dataset 
missing_hofers = df1[-51:]
df1 = df1[:-51]
df1.loc[df1['HOF'].isnull(),'HOF']=0
df1['HOF']=df1['HOF'].astype(int)
df1.shape

(21492, 61)

### Bring in SB Data

In [21]:
#get sb data and reformat
superbowls = pd.read_csv('Data/superbowls.csv')
sbwinners = superbowls[['Year','Team','SB Win']]
sblosers = superbowls[['Year','Loser','SB Loss']]
sblosers = sblosers.rename(columns={'Loser':'Team'})

df1 = df1.merge(sbwinners,'left')
df1 = df1.merge(sblosers,'left')
df1 = df1.fillna(0)
df1.shape

(21492, 63)

In [22]:
df1.columns

Index(['Player Id', 'Name', 'Position', 'Year', 'Team', 'Games Played',
       'Passes Attempted', 'Passes Completed', 'Completion Percentage',
       'Pass Attempts Per Game', 'Passing Yards', 'Passing Yards Per Attempt',
       'Passing Yards Per Game', 'TD Passes', 'Percentage of TDs per Attempts',
       'Ints', 'Int Rate', 'Longest Pass', 'Passes Longer than 20 Yards',
       'Passes Longer than 40 Yards', 'Sacks', 'Sacked Yards Lost',
       'Passer Rating', 'Rushing Attempts', 'Rushing Attempts Per Game',
       'Rushing Yards', 'Yards Per Carry', 'Rushing Yards Per Game',
       'Rushing TDs', 'Longest Rushing Run', 'Rushing First Downs',
       'Percentage of Rushing First Downs', 'Rushing More Than 20 Yards',
       'Rushing More Than 40 Yards', 'Receptions', 'Receiving Yards',
       'Yards Per Reception', 'Receiving Yards Per Game', 'Longest Reception',
       'Receiving TDs', 'Receptions Longer than 20 Yards',
       'Receptions Longer than 40 Yards', 'First Down Reception

### Pivot to Career Stats

In [23]:
##pivot to change from yearly stats to career stats
career = df1.pivot_table(index=['Player Id','Name','Position'],aggfunc=sum)
career = career.reset_index()

#change a few columns to means instead of sum
df1['Year'] = df1['Year'].astype(int)
for c in ['Year','First Year','Last Year','HOF','5Year']:
    career[c] = (round(career[c] / career['Seasons'])).astype(int)
    


### Fix Missing HOFers data

In [24]:
#bring in career stats for missing hofers
missing_hofers1 = missing_hofers.copy()
missing_hofers1['Player Id'] = missing_hofers1['Name']
missing_hofers1 = missing_hofers1[['Player Id','Name','HOF']]
#merge stats from originall hof data
missing_hofers1 = missing_hofers1.merge(hof)
#only grab columns we want (the ones in our original df)
common_cols = list(set(df.columns).intersection(missing_hofers1))
missing_hofers1 = missing_hofers1[common_cols]
missing_hofers1['HOF']=1
###get missing hofers sb data
missingsb = pd.read_csv('Data/missinghofsb.csv')
missing_hofers1 = missing_hofers1.merge(missingsb,'left')
#make estimate for hof seasons played
missing_hofers1['Seasons'] = 8
#merge in the missing hofers, and fill nas with 0s
career = career.merge(missing_hofers1,'outer')
career = career.fillna(0)
career['HOF']=career['HOF'].astype(int)
#fix inflation numbers
recalc(career)
career['5Year'] = career['Year'].round().astype(int) //5 *5
career = fix_inf(df,career,'5Year','Passing Yards')
career = fix_inf(df,career,'5Year','Rushing Yards')
career = fix_inf(df,career,'5Year','Receiving Yards')
career = fix_inf(df,career,'5Year','TD Passes')
career = fix_inf(df,career,'5Year','Rushing TDs')
career = fix_inf(df,career,'5Year','Receiving TDs')

### Get MVP and SB MVP data

In [25]:
#get data
mvp = pd.read_csv('Data/mvps.csv')
#get sum of mvps per player
mvp = mvp.pivot_table(index='Name',aggfunc=sum).reset_index().drop('Year',axis=1)
career = career.merge(mvp,'left')
career.loc[career['MVP'].isnull(),'MVP']=0

#get data
sbmvp = pd.read_csv('Data/sbmvps.csv')
#get sum of sbmvps per player
sbmvp = sbmvp.pivot_table(index='Name',aggfunc=sum).reset_index().drop('Year',axis=1)
career = career.merge(sbmvp,'left')
career.loc[career['SB MVP'].isnull(),'SB MVP']=0
#fix dan marino & jim kelly
career.loc[career['Name']=='Marino, Dan','SB Loss'] = 1
career.loc[(career['Name']=='Kelly, Jim')&(career['Position']=='WR/TE'),'HOF']=0
career.shape

(4492, 52)

In [26]:
### Get gw drives
gwd = pd.read_csv('Data/gwd.csv')
career = career.merge(gwd,'left')
career = career.fillna(0)
pgwd = pd.read_csv('Data/pgwd.csv')
career = career.merge(pgwd,'left')
career = career.fillna(0)
career.shape


(4492, 54)

In [27]:
#remove hof tes
tes = hof['Name'][hof['Position']=='TE']
telist = []
for a in tes:
    ind = career[career['Name']==a].index[0]
    telist.append(ind)
career = career.drop(telist)

#change position to number (for model)
pos_dict = {'QB':0, 'TB':1, 'WR/TE':2}
career = career.replace({'Position':pos_dict})
career['Position'].astype('category')

career.to_csv('Data/career1.csv')
career.shape

(4484, 54)